# Week 4

Yay! It's week 4. Today's we'll keep things light. 

I've noticed that many of you are struggling a bit to keep up and still working on exercises from the previous weeks. Thus, this week we only have two components with no lectures and very little reading. 


## Overview

* An exercise on visualizing geodata using a different set of tools from the ones we played with during Lecture 2.
* Thinking about visualization, data quality, and binning. Why ***looking at the details of the data before applying fancy methods*** is often important.

## Part 1: Visualizing geo-data

It turns out that `plotly` (which we used during Week 2) is not the only way of working with geo-data. There are many different ways to go about it. (The hard-core PhD and PostDoc researchers in my group simply use matplotlib, since that provides more control. For an example of that kind of thing, check out [this one](https://towardsdatascience.com/visualizing-geospatial-data-in-python-e070374fe621).)

Today, we'll try another library for geodata called "[Folium](https://github.com/python-visualization/folium)". It's good for you all to try out a few different libraries - remember that data visualization and analysis in Python is all about the ability to use many different tools. 

The exercise below is based on the code illustrated in this nice [tutorial](https://www.kaggle.com/daveianhickey/how-to-folium-for-maps-heatmaps-time-data), so let us start by taking a look at that one.

*Reading*. Read through the following tutorial
 * "How to: Folium for maps, heatmaps & time data". Get it here: https://www.kaggle.com/daveianhickey/how-to-folium-for-maps-heatmaps-time-data
 * (Optional) There are also some nice tricks in "Spatial Visualizations and Analysis in Python with Folium". Read it here: https://towardsdatascience.com/data-101s-spatial-visualizations-and-analysis-in-python-with-folium-39730da2adf

In [34]:
import folium
from folium import plugins
from folium.plugins import HeatMap
import pandas as pd
import numpy as np

crime_data = pd.read_csv("Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv", parse_dates=['Date', 'Time'])

In [ ]:
sf_map = folium.Map(location=[37.76, -122.446747], zoom_start=13, tiles="Stamen Toner")
folium.Marker([37.77919, -122.41914], popup="City Hall").add_to(sf_map)
sf_map

In [ ]:
crime_type = 'DRUG/NARCOTIC'
drug_crime = crime_data.query('Category == @crime_type')
drug_crime = drug_crime[(drug_crime['Date'] > '2016-6-1') & (drug_crime['Date'] < '2016-7-31')].groupby(['X', 'Y']).size()

drug_map = folium.Map(location=[37.76, -122.446747], zoom_start=13, tiles="Stamen Toner")
for idx, _ in drug_crime.items():
    folium.Marker([idx[1], idx[0]]).add_to(drug_map)

drug_map

> *Exercise 1.1*: A new take on geospatial data. 
>
>A couple of weeks ago (Part 3 of Week 2), we worked with spacial data by using color-intensity of shapefiles to show the counts of certain crimes within those individual areas. Today, we look at studying geospatial data by plotting raw data points as well as heatmaps on top of actual maps.
> 
> * First start by plotting a map of San Francisco with a nice tight zoom. Simply use the command `folium.Map([lat, lon], zoom_start=13)`, where you'll have to look up San Francisco's longitude and latitude.
> * Next, use the the coordinates for SF City Hall `37.77919, -122.41914` to indicate its location on the map with a nice, pop-up enabled maker. (In the screenshot below, I used the black & white Stamen tiles, because they look cool).
> <img src="https://raw.githubusercontent.com/suneman/socialdata2022/main/files/city_hall_2022.png" alt="drawing" width="600"/>
> * Now, let's plot some more data (no need for pop-ups this time). Select a couple of months of data for `'DRUG/NARCOTIC'` and draw a little dot for each arrest for those two months. You could, for example, choose June-July 2016, but you can choose anything you like - the main concern is to not have too many points as this uses a lot of memory and makes Folium behave non-optimally. 
> We can call this kind of visualization a *point scatter plot*.

Ok. Time for a little break. Note that a nice thing about Folium is that you can zoom in and out of the maps.

> *Exercise 1.2*: Heatmaps.
> * Now, let's play with **heatmaps**. You can figure out the appropriate commands by grabbing code from the main [tutorial](https://www.kaggle.com/daveianhickey/how-to-folium-for-maps-heatmaps-time-data)) and modifying to suit your needs.
>    * To create your first heatmap, grab all arrests for the category `'SEX OFFENSES, NON FORCIBLE'` across all time. Play with parameters to get plots you like.
>    * Now, comment on the differences between scatter plots and heatmaps. 
>.      - What can you see using the scatter-plots that you can't see using the heatmaps? 
>.      - And *vice versa*: what does the heatmaps help you see that's difficult to distinguish in the scatter-plots?
>    * Play around with the various parameters for heatmaps. You can find a list here: https://python-visualization.github.io/folium/plugins.html
>    * Comment on the effect on the various parameters for the heatmaps. How do they change the picture? (at least talk about the `radius` and `blur`).
> For one combination of settings, my heatmap plot looks like this.
> <img src="https://raw.githubusercontent.com/suneman/socialdata2022/main/files/crime_hot_spot.png" alt="drawing" width="600"/>
>    * In that screenshot, I've (manually) highlighted a specific hotspot for this type of crime. Use your detective skills to find out what's going on in that building on the 800 block of Bryant street ... and explain in your own words. 

(*Fun fact*: I remembered the concentration of crime-counts discussed at the end of this exercise from when I did the course back in 2016. It popped up when I used a completely different framework for visualizing geodata called [`geoplotlib`](https://github.com/andrea-cuttone/geoplotlib). You can spot it if you go to that year's [lecture 2](https://nbviewer.jupyter.org/github/suneman/socialdataanalysis2016/blob/master/lectures/Week3.ipynb), exercise 4.)

In [ ]:
import matplotlib.pyplot as plt

crime_type = 'SEX OFFENSES, NON FORCIBLE'
arrest_types = ['ARREST, BOOKED', 'ARREST, CITED'] # it mentions arrests, but there are only 3, I don't think that's what she meant
sex_offenses = crime_data.query('Category == @crime_type') #[crime_data.Resolution.isin(arrest_types)]

sex_offenses_map = folium.Map(location=[37.76, -122.446747], zoom_start=13, tiles="Stamen Toner")
sex_offenses_data = [[row['Y'], row['X']] for _, row in sex_offenses.iterrows()]

HeatMap(sex_offenses_data).add_to(sex_offenses_map)
sex_offenses_map

In [ ]:
scatter_data = list(zip(*sex_offenses_data))
fig, axs = plt.subplots(figsize=(10,10))
axs.scatter(scatter_data[1], scatter_data[0])
plt.show()

In [98]:
"""On the heatmap it is difficult to see how many occurrences are responsible for that heat at first glance.
   On the scatter plot there could be overlapping points which we can't see at first glance. Also the heatmap helps us tell a little bit of the sorroundings of the crime"""

"On the heatmap it is difficult to see how many occurrences are responsible for that heat at first glance.\n   On the scatter plot there could be overlapping points which we can't see at first glance. Also the heatmap helps us tell a little bit of the sorroundings of the crime"

For the final element of working with heatmaps, let's now use the cool Folium functionality `HeatMapWithTime` to create a visualization of how the patterns of your favorite crime-type changes over time.

> *Exercise 1.3*: Heatmap movies. This exercise is a bit more independent than above - you get to make all the choices.
> * Start by choosing your favorite crimetype. Prefereably one with spatial patterns that change over time (use your data-exploration from the previous lectures to choose a good one).
> * Now, choose a time-resolution. You could plot daily, weekly, monthly datasets to plot in your movie. Again the goal is to find interesting temporal patterns to display. We want at least 20 frames though.
> * Create the movie using `HeatMapWithTime`.
> * Comment on your results: 
>   - What patterns does your movie reveal?
>   - Motivate/explain the reasoning behind your choice of crimetype and time-resolution. 

In [142]:
favorite_crime = 'DRIVING UNDER THE INFLUENCE'
favorite_crime = crime_data[crime_data.Category == favorite_crime]
favorite_crime = favorite_crime.dropna(axis=0, subset=['Y','X', 'Time', 'Date'])
favorite_crime_map = folium.Map(location=[37.76, -122.446747], zoom_start=13, tiles="Stamen Toner")

In [143]:
favorite_crime = [[[row['Y'], row['X']] for _, row in favorite_crime[favorite_crime.Time.dt.hour == hour].iterrows()] for hour in range(0,24)]
favorite_crime_heatmap = plugins.HeatMapWithTime(favorite_crime, auto_play=True, max_opacity=0.8)
favorite_crime_heatmap.add_to(favorite_crime_map)

favorite_crime_map

In [ ]:
"""
Driving under the influence over a 24 hour period:
    * during the day (6am to 4pm) there are way less cases than in the afternoon/night
    * number of cases peak during the nigh and are most prelevant in Tenderloin and the Geary Boulevard
    * from 4pm there are an increasing amount of cases around the pubs in Tenderloin, up until 5am
    * from 23pm to 6am there are an increasing amount of cases on the Geary Boulevard, 
    which leads to Richmond district, where people leave -> people are driving home from a night out in town
"""

In [ ]:
"""
We had a hypothesis that during the day there are way less people driving under the influence than in the afternoon/night, 
and that in the city center where pubs and night clubs are there are 
"""

## Part 2: Errors in the data. The importance of looking at raw (or close to raw) data.

We started the course by plotting simple histogram and bar plots that showed a lot of cool patterns. But sometimes the binning can hide imprecision, irregularity, and simple errors in the data that could be misleading. In the work we've done so far, we've already come across at least three examples of this in the SF data. 

1. In the temporal activity for `PROSTITUTION` something surprising is going on on Thursday. Remind yourself [**here**](https://raw.githubusercontent.com/suneman/socialdata2022/main/files/prostitution.png), where I've highlighted the phenomenon I'm talking about.
2. When we investigated the details of how the timestamps are recorded using jitter-plots, we saw that many more crimes were recorded e.g. on the hour, 15 minutes past the hour, and to a lesser in whole increments of 10 minutes. Crimes didn't appear to be recorded as frequently in between those round numbers. Remind yourself [**here**](https://raw.githubusercontent.com/suneman/socialdata2022/main/files/jitter.png), where I've highlighted the phenomenon I'm talking about.
3. And, today we saw that the Hall of Justice seemed to be an unlikely hotspot for sex offences. Remind yourself [**here**](https://raw.githubusercontent.com/suneman/socialdata2022/main/files/crime_hot_spot.png).

> *Exercise 2*: Data errors. The data errors we discovered above become difficult to notice when we aggregate data (and when we calculate mean values, as well as statistics more generally). Thus, when we visualize, errors become difficult to notice when binning the data. We explore this process in the exercise below.
>
>This last exercise for today has two parts:
> * In each of the examples above, describe in your own words how the data-errors I call attention to above can bias the binned versions of the data. Also, briefly mention how not noticing these errors can result in misconceptions about the underlying patterns of what's going on in San Francisco (and our modeling).
> * Find your own example of human noise in the data and visualize it.